# STATISTICAL MACHINE LEARNING (WS2022). COMPUTER LAB

### Independent linear multi-class classifier

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join


# load single example
def load_example(img_path):

    Y = img_path[img_path.rfind('_') + 1:-4]

    img = Image.open(img_path)
    img_mat = np.asarray(img)

    n_letters = len(Y)
    im_height = int(img_mat.shape[0])
    im_width = int(img_mat.shape[1] / n_letters)
    n_pixels = im_height * im_width

    X = np.zeros([int(n_pixels + n_pixels * (n_pixels - 1) / 2), n_letters])
    for i in range(n_letters):

        # single letter
        letter = img_mat[:, i * im_width:(i + 1) * im_width] / 255

        # compute features
        x = letter.flatten()
        X[0:len(x), i] = x
        cnt = n_pixels
        for j in range(0, n_pixels - 1):
            for k in range(j + 1, n_pixels):
                X[cnt, i] = x[j] * x[k]
                cnt = cnt + 1

        X[:, i] = X[:, i] / np.linalg.norm(X[:, i])

    return X, Y, img


# load all examples from a folder
def load_examples(image_folder):

    files = [f for f in listdir(image_folder) if isfile(join(image_folder, f))]

    X = []
    Y = []
    img = []
    for file in listdir(image_folder):
        path = join(image_folder, file)
        if isfile(path):

            X_, Y_, img_ = load_example(path)
            X.append(X_)
            Y.append(Y_)
            img.append(img_)

    return X, Y, img


def n2l(num):
    # number to letter
    return chr(97 + int(num))


def l2n(let):
    # letter to number
    return ord(let) - ord('a')


def compute_idxs_ind(l_y, X):
    n = l_y
    if type(l_y) is np.str_ and len(l_y) == 1:
        n = l2n(l_y)
    # n - the number of the letter in alphabet
    idx_start = n * X + n
    idx_end = n * X + X + n
    return idx_start, idx_end


def train_independent_linear_classifier(trn_X, trn_Y, N):
    counter = 0
    W = np.zeros((N * trn_X[0].shape[0] + N))
    X = trn_X[0].shape[0]
    while True:
        print(counter)
        counter += 1
        missclass = 0
        for i in range(len(trn_X)):
            # trn_X[i] is a word
            for letter in range(trn_X[i].shape[1]):
                # trn_X[i][letter] is a letter
                y_correct = l2n(str(trn_Y[i][letter]))
                y_hat = np.zeros(N)
                for y in range(N):
                    # y - guess letter
                    b, e = compute_idxs_ind(y, X)
                    tmp_vec = np.append(trn_X[i][:, letter], [1])
                    y_hat[y] = tmp_vec @ W[b:e + 1]
                maxx = np.argmax(y_hat)
                if maxx != y_correct:
                    b1, e1 = compute_idxs_ind(y_correct, X)
                    b2, e2 = compute_idxs_ind(maxx, X)
                    W[b1: e1] += trn_X[i][:, letter]
                    W[e1] += 1
                    W[b2: e2] -= trn_X[i][:, letter]
                    W[e2] -= 1
                    missclass += 1
                    print(".", end='')

        if missclass == 0:
            return W


def test_independent_linear_classifier(tst_X, tst_Y, N, W):
    l_X_ = tst_X[0].shape[0]
    error_char_sum = 0
    error_seq_sum = 0
    char_counter = 0
    for i in range(len(tst_X)):
        # trn_X[i] is a word
        res_word = ""
        for letter in range(tst_X[i].shape[1]):
            char_counter += 1
            # trn_X[i][letter] is a letter
            y_correct = l2n(str(tst_Y[i][letter]))
            y_hat = np.zeros(N)
            for y in range(N):
                # y - guess letter
                b, e = compute_idxs_ind(y, l_X_)
                tmp_vec = np.append(tst_X[i][:, letter], [1])
                y_hat[y] = tmp_vec @ W[b:e + 1]
            maxx = np.argmax(y_hat)
            res_word += n2l(maxx)
            if maxx != y_correct:
                error_char_sum += 1
        if res_word != tst_Y[i]:
            # print(res_word, "!=", tst_Y[i])
            error_seq_sum += 1

    return error_seq_sum / len(tst_X), error_char_sum / char_counter



In [ ]:
# load training examples
trn_X, trn_Y, trn_img = load_examples('ocr_names_images/trn')

# load testing examples
tst_X, tst_Y, tst_img = load_examples('ocr_names_images/tst')



### Linear structured classifier modeling pair-wise dependency 

In [ ]:
N = 26  # number of letters in alphabet

In [ ]:
X = trn_X[0].shape[0]
SIZE = (X + N + 1) * N


def fn_q(l_x, l_W, l_y):
    l_beg, b_idx, l_end = compute_idxs_pairs(l_y, X, N)
    tmp = l_x @ l_W[l_beg:b_idx]
    return tmp + l_W[b_idx]


def fn_g_idx(y1: int, y2: int):
    return y1 * (X + N + 1) + X + 1 + y2


def fn_g(y1: int, y2: int, l_W):
    # the letter y1 after y2
    return l_W[fn_g_idx(y1, y2)]


def fn_f(l_W, l_F_mat, l_Y_mat, l_trn_X):
    for i_L in range(N):
        l_F_mat[i_L, 0] = fn_q(l_trn_X[:, 0], l_W, i_L)

    for i_L in range(1, l_trn_X.shape[1]):  # i_L e letters of a word
        for i_y in range(N):  # i_y e Alphabet
            l_f = np.array([l_F_mat[k][i_L - 1] + fn_g(k, i_y, l_W) for k in range(N)])
            l_Y_mat[i_y, i_L - 1] = np.argmax(l_f)
            l_F_mat[i_y, i_L] = fn_q(l_trn_X[:, i_L], l_W, i_y) + np.max(l_f)


def compute_idxs_pairs(l_y, X_feature_size: int, alphabet_size: int):
    n = l_y
    if type(l_y) is np.str_ and len(l_y) == 1:
        n = l2n(l_y)
    # n - the number of the letter in alphabet
    idx_start = n * (X_feature_size + 1 + alphabet_size)
    idx_b = idx_start + X_feature_size
    idx_end = n * (X_feature_size + 1 + alphabet_size) + X_feature_size + alphabet_size + 1
    return idx_start, idx_b, idx_end


# W structure: [X(a), b(a), g(a a)...g(a z); .....; X(z), b(z), g(z a)...g(z, z)]
def phi(l_x, l_y):
    res = np.zeros(SIZE)
    for i_ in range(len(l_y)):
        l_beg, b_idx, l_end = compute_idxs_pairs(l2n(l_y[i_]), X, N)
        res[l_beg:b_idx] += l_x[:, i_]
        res[b_idx] += 1

    for i_ in range(1, len(l_y)):
        res[fn_g_idx(l2n(l_y[i_ - 1]), l2n(l_y[i_]))] += 1

    return res


def train_linear_pairwise_classifier(trn_X, trn_Y, N):
    W = np.zeros(SIZE)
    counter = 0
    while True:
        print(counter)
        counter += 1
        missclass = 0
        for i in range(len(trn_X)):
            # trn_X[i] is a word
            tmp_idx = trn_X[i].shape[1]
            F_mat = np.zeros((N, tmp_idx))
            Y_mat = np.zeros((N, tmp_idx - 1))

            fn_f(W, F_mat, Y_mat, trn_X[i])

            guessed_word = n2l(np.argmax(F_mat[:, -1]))
            for j in range(trn_X[i].shape[1] - 1):
                guessed_word += n2l(Y_mat[l2n(guessed_word[-1])][-1 - j])

            guessed_word = guessed_word[::-1]

            if guessed_word != trn_Y[i]:
                # update the W
                W += phi(trn_X[i], trn_Y[i]) - phi(trn_X[i], guessed_word)
                missclass += 1
                print(".", end='')
        if missclass == 0:
            return W


def test_linear_pairwise_classifier(trn_X, trn_Y, N, W):
    X = tst_X[0].shape[0]
    error_char_sum = 0
    error_seq_sum = 0
    char_counter = 0
    for i in range(len(trn_Y)):
        # trn_X[i] is a word
        tmp_idx = trn_X[i].shape[1]
        F_mat = np.zeros((N, tmp_idx))
        Y_mat = np.zeros((N, tmp_idx - 1))
        fn_f(W, F_mat, Y_mat, trn_X[i])
        guessed_word = n2l(np.argmax(F_mat[:, -1]))
        for j in range(trn_X[i].shape[1] - 1):
            guessed_word += n2l(Y_mat[l2n(guessed_word[-1])][-1 - j])
        guessed_word = guessed_word[::-1]
        for c in range(len(guessed_word)):
            char_counter += 1
        if guessed_word != trn_Y[i]:
            for c in range(len(guessed_word)):
                char_counter += 1
                if guessed_word[c] != trn_Y[i][c]:
                    error_char_sum += 1
            error_seq_sum += 1
    return error_seq_sum / len(tst_X), error_char_sum / char_counter

In [ ]:
K = 0
print(f"#features={trn_X[K].shape[0]}")
print(f"#features={trn_X[K].shape[1]}")
print(f"#trn examples={len(trn_X)}")
print(f"#tst examples={len(tst_X)}")

# show the first testing example
# for i in range(10):
#     plt.figure()
#     plt.imshow( trn_img[i], cmap='Greys')
#     plt.title( trn_Y[i] )
# plt.figure()
# plt.imshow( trn_img[K], cmap='Greys')
# plt.title( trn_Y[K] )

# for i in range(trn_X[K].shape[1]):
#     plt.figure()
#     plt.plot( trn_X[K][:,i])
#     plt.title(f"features of character {trn_Y[K][i]}")


In [ ]:
# W = train_independent_linear_classifier(trn_X, trn_Y, N)
# print(W.shape)
# e1, e2 = test_independent_linear_classifier(trn_X, trn_Y, N, W)
# print("train_set_errors", e1, e2)  # 0.0 0.0
# err1, err2 = test_independent_linear_classifier(tst_X, tst_Y, N, W)
# print("test_set_errors: ", err1, err2)  # 0.706 0.26392823418319167

In [ ]:
W = train_linear_pairwise_classifier(trn_X, trn_Y, N)
e1, e2 = test_linear_pairwise_classifier(trn_X, trn_Y, N, W)
print("train_set_errors", e1, e2) # 0.0 0.0
err1, err2 = test_linear_pairwise_classifier(tst_X, tst_Y, N, W)
print("test_set_errors: ", err1, err2) # 0.118 0.04643449419568822